# Iterative Closest Point (ICP)

In [18]:
# imports
import numpy as np
import numpy.linalg

## Data setup

Rotation matrix:
$
\begin{bmatrix}
\cos(\theta) & -\sin(\theta) & t_x \\
\sin(\theta) & \cos(\theta) & t_y \\
0 & 0 & 1 \\
\end{bmatrix}
$

In [68]:
import random

def randomTransformation2D(maxRot: float = 30, maxTrans: float = 1):
    """
    Generate random 2D rigid transformation centered at the origin and with
    an angle in [-maxRot; maxRot] and a translation in lower than maxTrans

    Parameters
    ----------
    maxRot : float
        maximum rotation in degrees
    maxTrans : float
        maximum norm for the translation vector

    Returns
    -------
    rot : np.array
        Generated random 2d rigid transformation
    """

    rot = np.zeros((3, 3), dtype=float)

    # Angle
    angle = random.uniform(-maxRot, maxRot)
    # Convert degrees to radian
    angle = angle * np.pi / 180
    rot[0:2, 0:2] = np.array([[np.cos(angle), -np.sin(angle)],
                              [np.sin(angle), np.cos(angle)]])

    # Translation
    # Arbitrary interval
    translation = np.array([random.uniform(-maxTrans, maxTrans),
                            random.uniform(-maxTrans, maxTrans)])
    # Normalize vector
    translation /= np.linalg.norm(translation, ord=2)
    # Random norm lower than maxNorm
    translation *= random.uniform(0, maxTrans)
    rot[0:2, 2] = translation
    rot[2, 2] = 1

    return rot

randomTransformation2D()

-0.1771328586096224


array([[ 0.98435295,  0.17620802,  0.19328163],
       [-0.17620802,  0.98435295,  0.88399162],
       [ 0.        ,  0.        ,  1.        ]])

In [ ]:
## Rotation 3D
## Axe de rotation aleatoire

## Implementation

In [ ]:
from icp.icp import icp # Import OK

## Test

## Variants